# Fanfooty Supercoach Prediction Model
Selecting your supercoach team is hard... this model is designed as a tool to assist supercoach users in selecting their team week-to-week.

This model uses rolling stats, opposition data, and past injury data to provide a predicted score for matches.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 30)

### Get Fanfooty Data
This data was scraped using scrape-fanfooty.pynb in this directory

In [2]:
df_fanfooty_player_raw = pd.read_csv('exports/scrape_20200712-180836/fanfooty_match_data_20200712-180836.csv', error_bad_lines=False)
df_fanfooty_player_raw = df_fanfooty_player_raw.loc[df_fanfooty_player_raw['SC'] != '-']
df_fanfooty_player_raw['SC'] = df_fanfooty_player_raw['SC'].astype('int64')
df_fanfooty_player_raw

b'Skipping line 89678: expected 49 fields, saw 50\nSkipping line 89679: expected 49 fields, saw 50\nSkipping line 89680: expected 49 fields, saw 50\nSkipping line 89681: expected 49 fields, saw 50\nSkipping line 89682: expected 49 fields, saw 50\nSkipping line 89683: expected 49 fields, saw 50\nSkipping line 89684: expected 49 fields, saw 50\nSkipping line 89685: expected 49 fields, saw 50\nSkipping line 89686: expected 49 fields, saw 50\nSkipping line 89687: expected 49 fields, saw 50\nSkipping line 89688: expected 49 fields, saw 50\nSkipping line 89689: expected 49 fields, saw 50\nSkipping line 89690: expected 49 fields, saw 50\nSkipping line 89691: expected 49 fields, saw 50\nSkipping line 89692: expected 49 fields, saw 50\nSkipping line 89693: expected 49 fields, saw 50\nSkipping line 89694: expected 49 fields, saw 50\nSkipping line 89695: expected 49 fields, saw 50\nSkipping line 89696: expected 49 fields, saw 50\nSkipping line 89697: expected 49 fields, saw 50\nSkipping line 8969

,Fanfooty Match ID,Fanfooty Match URL,Round,Year,Player ID,First Name,Surname,Team,null,DT,SC,null2,null3,null4,Kicks,Handballs,Marks,Tackles,Hitouts,Frees for,Frees against,Goals,Behinds,Not sure,Tag,Tag Notes,Tag 2,Tag 2 Notes,null5,null6,null7,null8,Position,Jumper Number,null9,null10,null11,DT own %,SC own %,AF own %,null12,AF Breakeven,null13,Contested Possessions,Clearances,Clangers,Disposal efficiency,Time on ground,Metres gained
0,3425,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010,990020.0,Andrew,Embley,WC,30,111,98,144,79,112,20,8,1,6,1,1,0,1,0,Full Time,gun,Dempsey going with him... %s from %O and %T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3425,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010,230254.0,Adam,Selwood,WC,50,107,107,143,79,108,10,9,4,11,0,3,2,1,0,Full Time,hot,Tagged by Lonergan... %D and %M with %T plus %s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3425,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010,200112.0,Dean,Cox,WC,27,99,118,114,88,106,9,10,2,2,30,4,1,1,1,Full Time,news,%H and %P with %s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3425,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010,240016.0,Beau,Waters,WC,26,98,84,130,79,117,15,13,5,6,0,0,4,0,0,Full Time,news,%P and %M with %F... clangers and FA dampening...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3425,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010,261911.0,Brad,Ebert,WC,26,94,109,121,70,96,12,9,3,6,0,1,0,1,0,Full Time,news,Matched up on Winderlich... %D and %T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90859,7353,http://live.fanfooty.com.au/game/matchcentre.h...,R6,2020,997100.0,Will,Hayward,SY,4,44,65,24,38,51,8,5,4,0,0,1,1,0,0,Full Time,guard,%O and %M... Starting in defence,NaN,NaN,NaN,NaN,NaN,NaN,Forward,9.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,3.0,0.0,2.0,84.0,86.0,172.0
90860,7353,http://live.fanfooty.com.au/game/matchcentre.h...,R6,2020,290722.0,Sam,Gray,SY,4,42,43,40,32,42,4,3,4,3,0,0,0,0,0,Full Time,wing,%O and %M plus %T... Playing a HFF role,NaN,NaN,NaN,NaN,NaN,NaN,Midfielder,15.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,2.0,0.0,0.0,85.0,81.0,70.0
90861,7353,http://live.fanfooty.com.au/game/matchcentre.h...,R6,2020,1008080.0,Ryley,Stoddart,SY,4,38,47,32,30,42,4,6,2,2,0,0,0,0,0,Full Time,guard,%O and %M plus %T... Rotating in defence,NaN,NaN,NaN,NaN,NaN,NaN,Back,33.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,4.0,0.0,2.0,70.0,78.0,87.0
90862,7353,http://live.fanfooty.com.au/game/matchcentre.h...,R6,2020,1012014.0,Chad,Warner,SY,3,29,38,23,25,36,4,4,2,2,0,1,2,0,0,Full Time,rookie,First game... %D and %M with %T,shovel,Rotating in midfield,NaN,NaN,NaN,NaN,Midfielder,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,4.0,0.0,3.0,75.0,78.0,114.0


### Identify when players were injured during a match
Fanfooty has amazing "tags" that can be used to identify when a player has been injured during a match

If they have certain tags (e.g. concussed) and score below 80 supercoach points, they are judged as injured.

In [3]:
injured_tags = [
    'sore',
    'injured',
    'longterminjured',
    'concussed',
    'heart'
]

def get_injured_status(row):
    if (row['Tag'] in injured_tags or row['Tag 2'] in injured_tags) and row['SC'] < 80:
        return True
    else:
        return False

df_fanfooty_player_raw['Injured'] = df_fanfooty_player_raw.apply(lambda row: get_injured_status(row), axis=1)

### Get Fanfooty fixture data
This data was scraped using scrape-fanfooty.pynb in this directory

In [4]:
df_fixture = pd.read_csv("exports/scrape_20200712-180836/fanfooty_fixture.csv", index_col=0)
df_fixture

,FanFooty draw ID,year,competition,round,gametime (AET),day,home team,away team,ground,timeslot,TV coverage,home supergoals,home goals,home behinds,home points,away supergoals,away goals,away behinds,away points,match status
0,1006.0,1993.0,HA,1.0,1993-03-26 20:08:00,Friday,Western Bulldogs,Collingwood,MCG,N,NaN,NaN,13.0,17.0,95.0,NaN,17.0,13.0,115.0,Full Time
1,1000.0,1993.0,HA,1.0,1993-03-27 14:00:00,Saturday,North Melbourne,Brisbane Bears,MCG,D,NaN,NaN,24.0,22.0,166.0,NaN,22.0,11.0,143.0,Full Time
2,1001.0,1993.0,HA,1.0,1993-03-27 14:00:00,Saturday,Carlton,Fitzroy,Princes Park,D,NaN,NaN,17.0,10.0,112.0,NaN,17.0,16.0,118.0,Full Time
3,1002.0,1993.0,HA,1.0,1993-03-27 14:00:00,Saturday,Hawthorn,Melbourne,Waverley,D,NaN,NaN,13.0,15.0,93.0,NaN,11.0,4.0,70.0,Full Time
4,1005.0,1993.0,HA,1.0,1993-03-27 14:08:00,Saturday,Geelong,St Kilda,Kardinia,D,NaN,NaN,20.0,16.0,136.0,NaN,16.0,16.0,112.0,Full Time
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5650,7441.0,2020.0,HA,17.0,2020-09-26 19:25:00,Saturday,Brisbane Lions,Geelong,Gabba,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5651,7442.0,2020.0,HA,17.0,2020-09-27 13:10:00,Sunday,Port Adelaide,Gold Coast,Adelaide,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5652,7443.0,2020.0,HA,17.0,2020-09-27 15:20:00,Sunday,Carlton,Sydney,Docklands,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5653,7444.0,2020.0,HA,17.0,2020-09-27 17:20:00,Sunday,West Coast,Adelaide,Perth,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Get the total SuperCoach and AFL Fantasy scores for each team, for every match

In [5]:
# Create a summary to get the total SC and AF points for each match
df_match_summary = pd.pivot_table(df_fanfooty_player_raw, index=['Fanfooty Match ID'], values=['SC'], columns=['Team'], aggfunc=np.sum)
# df_match_summary = df_match_summary.reset_index()
# df_match_summary.columns = df_match_summary.columns.to_series().str.join('_')
# df_match_summary = df_match_summary.rename(columns={'Match_id_': 'Match_id'})
df_match_summary

SC                                                  \
Team                   AD      BL      CA      CO      ES  FR      GC  GE   
Fanfooty Match ID                                                           
3425                  NaN     NaN     NaN     NaN  1568.0 NaN     NaN NaN   
3426                  NaN     NaN     NaN     NaN     NaN NaN     NaN NaN   
3427               1513.0     NaN  1826.0     NaN     NaN NaN     NaN NaN   
3428                  NaN     NaN     NaN  1873.0     NaN NaN     NaN NaN   
3429                  NaN  1781.0     NaN     NaN     NaN NaN     NaN NaN   
...                   ...     ...     ...     ...     ...  ..     ...  ..   
7349               1605.0     NaN     NaN     NaN     NaN NaN     NaN NaN   
7350                  NaN     NaN     NaN     NaN     NaN NaN  1658.0 NaN   
7351                  NaN     NaN     NaN     NaN  1812.0 NaN     NaN NaN   
7352                  NaN     NaN     NaN     NaN     NaN NaN     NaN NaN   
7353                  NaN     NaN     NaN     NaN     NaN NaN     NaN NaN   

                                                                               \
Team                   HW      ME      NM      PA      RI  SK      SY      WB   
Fanfooty Match ID                                                               
3425                  NaN     NaN     NaN     NaN     NaN NaN     NaN     NaN   
3426                  NaN     NaN  1504.0     NaN     NaN NaN  1797.0     NaN   
3427                  NaN     NaN     NaN     NaN     NaN NaN     NaN     NaN   
3428               1447.0     NaN     NaN     NaN     NaN NaN     NaN     NaN   
3429                  NaN     NaN     NaN     NaN     NaN NaN     NaN  1545.0   
...                   ...     ...     ...     ...     ...  ..     ...     ...   
7349                  NaN     NaN     NaN     NaN     NaN NaN     NaN     NaN   
7350                  NaN  1641.0     NaN     NaN     NaN NaN     NaN     NaN   
7351                  NaN     NaN  1489.0     NaN     NaN NaN     NaN     NaN   
7352                  NaN     NaN     NaN  1657.0     NaN NaN     NaN     NaN   
7353                  NaN     NaN     NaN     NaN  1699.0 NaN  1602.0     NaN   

                                   
Team                   WC      WS  
Fanfooty Match ID                  
3425               1739.0     NaN  
3426                  NaN     NaN  
3427                  NaN     NaN  
3428                  NaN     NaN  
3429                  NaN     NaN  
...                   ...     ...  
7349               1692.0     NaN  
7350                  NaN     NaN  
7351                  NaN     NaN  
7352                  NaN  1638.0  
7353                  NaN     NaN  

[2062 rows x 18 columns]